In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics

## 1. Reading data and removing no Sales items

In [2]:
dirpath = '/Users/parulgaba/Desktop/Capstone-Ethos/ConfidentialData/csvdata/'
data_path = '/Users/parulgaba/Desktop/Capstone-Ethos/ethos-retail-model/data/'
filename = data_path + 'regression_data/' + 'aggregated_summary_store_type_12_weeks.csv'

#filename = 'D:\\Backup\\ISB CBA\\Capstone\\summary data\\aggregated_summary_store_type_12_weeks.csv'

chunksize = 10 ** 5
rows=0
summary_df = pd.DataFrame()
for chunk in pd.read_csv(filename, chunksize=chunksize):
    summary_df=pd.concat([summary_df,chunk])
    rows+=chunk.shape[0]
    
summary_df.fillna(0)
print(summary_df.shape)
print (rows)

(476711, 47)
476711


In [4]:
#removing items with no sales in the 3 year period
items_no_sales = summary_df.groupby(['item_no']).agg({'sales_quantity':'sum'}).reset_index()
unique_item_no_sales = items_no_sales[items_no_sales['sales_quantity'] == 0]['item_no'].unique()
summary_df = summary_df[~summary_df['item_no'].isin(unique_item_no_sales)]
print("Unique items removed with no sales at all for all 3 three years : " + str(len(unique_item_no_sales)))

Unique items removed with no sales at all for all 3 three years : 0


In [5]:
summary_df['stock_prevailing_mrp'] = summary_df['stock_prevailing_mrp'].div(10000)
summary_df['billing'] = summary_df['billing'].div(10000)

In [6]:
summary_df.shape

(438196, 47)

In [7]:
summary_df['item_no']=summary_df['item_no'].astype(str)
summary_df['period']=summary_df['period'].astype(int)
summary_df['case_shape']=summary_df['case_shape'].astype(str)


## 2. Pre processing data

###      2a. Performing Item Pareto

To combine all SKUs of a specific brand that cummulatively account for ~5% or less by Sales billings into a new SKU called "Others".

In [8]:
#2(a)(i) Code for item pareto

pareto =0.05
import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')

# Create a new df with Other items

summary_item_pareto_final = pd.DataFrame()
lst = []
item_mapping_list = []

item_cols = ['case_size_range', 'gender','movement', 'material', 'dial_color', 'strap_type', 'strap_color','precious_stone', 'glass', 'case_shape', 'watch_type']
brands = summary_df['brand'].unique()
for brand in brands:
    items_combined_df = pd.DataFrame()

    summary_by_brand_df = summary_df[summary_df['brand'] == brand]
    item_series = summary_by_brand_df.fillna(0).groupby('item_no').agg({'billing':'sum'}).sort_values('billing',ascending=False)
    items_combined_df = pd.concat([items_combined_df, item_series])
    
    mask=items_combined_df.cumsum()/items_combined_df.sum()>(1-pareto)
    mask=mask.iloc[:,0]
    
    levels=len(summary_by_brand_df['item_no'].unique())
    
    if levels > 100:
        summary_by_brand_df['brand'] = np.where(summary_by_brand_df['item_no'].isin(item_series[mask].index),'Others',summary_by_brand_df['brand'])         
        summary_by_brand_df['item_no'] = np.where(summary_by_brand_df['item_no'].isin(item_series[mask].index),'Others',summary_by_brand_df['item_no'])
        item_mapping_list = item_mapping_list + item_series[mask].index.to_list()
        for col in item_cols:
            if summary_df[col].mode()[0] == 'NaN':
                print(brand)
                print (col)
                print(summary_df[col].mode()[0])
            summary_by_brand_df[col] = np.where(summary_by_brand_df['item_no'].isin(item_series[mask].index),summary_df[col].mode()[0],summary_by_brand_df[col])
    
    new_levels=len(summary_by_brand_df['item_no'].unique())
    
    freq=summary_by_brand_df['item_no'].value_counts()/summary_by_brand_df['item_no'].value_counts().sum()*100
    freq=freq.round(2)
    
    sale_qty=summary_by_brand_df.groupby(['item_no']).agg({'sales_quantity':'sum'}).sort_values('sales_quantity',ascending=False)
    sale_qty=sale_qty/sale_qty.sum()*100
    sale_qty=sale_qty.round(2)
    
    try:
        Other_Sales_Qty=sale_qty['sales_quantity']['Others']
    except:
        Other_Sales_Qty=0
    bill=summary_by_brand_df.groupby(['item_no']).agg({'billing':'sum'}).sort_values('billing',ascending=False)
    bill=bill/bill.sum()*100
    bill=bill.round(2)
    try:
        Other_bill=bill['billing']['Others']
    except:
        Other_bill=0
        
    lst.append([brand, levels, new_levels,Other_bill,Other_Sales_Qty])
    
    cols=['Brand', 'Orig SKU count', 'New SKU count', 'Other%(Billing)', 'Other%(Sales Qty)']
    item_pareto_summary = pd.DataFrame(lst, columns=cols)
    item_pareto_summary=item_pareto_summary.set_index("Brand")
    
    summary_item_pareto_final = pd.concat([summary_item_pareto_final, summary_by_brand_df])
#items_combined_df.head(5)

In [9]:
item_pareto_summary.sort_values(by = ['Other%(Sales Qty)'], ascending=False).head()

,Orig SKU count,New SKU count,Other%(Billing),Other%(Sales Qty)
Brand,,,,
B046,156,119,5.10,22.17
B124,120,84,5.14,19.76
B023,197,138,5.11,19.24
B094,591,409,5.02,17.50
B014,391,258,5.07,13.97


In [10]:
summary_item_pareto_final.head()

,period,item_no,state,store_type,brand,store_location,city_type,region,quantity,purchase_quantity,...,movement,material,dial_color,strap_type,strap_color,precious_stone,glass,case_shape,watch_type,area_code
0,1,5103485,ST12,Summit,B027,Mall,Tier1,West,1,0,...,quartz,steel,white,steel,silver,no,sapphire crystal,square,analog,3
587,10,5161506,ST09,Summit,B027,Mall,Tier1,South,1,0,...,automatic,steel & rose gold,black,rubber,black,no,sapphire crystal,round,analog,6
794,7,5168340,ST09,Summit,B027,Mall,Tier1,South,1,0,...,automatic,steel,silver,leather,black,no,sapphire crystal,round,analog,6
1616,5,5182086,ST02,Ethos,B027,High Street,Tier2,North,1,0,...,automatic,steel,silver,leather,black,no,sapphire crystal,rectangle,analog,1
2534,5,5119075,ST03,Summit,B027,Mall,Tier1,North,0,1,...,automatic,steel,silver,steel,silver,no,sapphire crystal,round,analog,2


In [11]:
#2(a)(ii) Aggregating all "Other" SKUS into SKU per brand

summarize_method = {
        'brand' :'first', 'store_location' :'first',
       'city_type' :'first', 'region' :'first', 'quantity' :'mean', 'purchase_quantity' :'mean',
       'transfer_quantity' :'mean', 'available_quantity' :'mean', 'sales_quantity' :'mean',
       'purchase_cost_amount' :'mean', 'purchase_mrp' :'mean', 'purchase_date' :'first',
       'stock_prevailing_mrp' :'mean', 'store_in' :'first', 'product_group_code' :'first',
       'transfer_cost_amount' :'mean', 'sales_department' :'first', 'days_to_sell' :'mean',
       'num_of_customers' :'mean', 'total_price' :'mean', 'line_discount' :'mean', 'crm_line_discount' :'mean',
       'discount' :'mean', 'tax' :'mean', 'cost' :'mean', 'billing' :'mean', 'contribution' :'mean', 'trade_incentive' :'mean',
       'trade_incentive_value' :'mean', 'total_contribution' :'mean', 'case_size' :'mean',
       'case_size_range' :'first', 'gender' :'first', 'movement' :'first', 'material' :'first', 'dial_color' :'first',
       'strap_type' :'first', 'strap_color' :'first', 'precious_stone' :'first', 'glass' :'first', 'case_shape' :'first',
       'watch_type' :'first', 'area_code' :'first'
}


sales_sum_df = summary_item_pareto_final.groupby(['store_type','item_no','period','state']).agg(summarize_method).reset_index()
sales_sum_df.shape

(412140, 47)

### 2b. Adding market share column {log S/So}

In [12]:
#2(B)(i) - Adding log S/So

#reading market shares
market_share=pd.read_excel(data_path + "market_share_encoded.xlsx", header=0,index_col=0)

#computing market size for each state-period
market_sizes=sales_sum_df.groupby(['state','period']).agg({'sales_quantity':'sum'})
market_sizes=market_sizes.reset_index()
market_sizes=pd.merge(market_sizes,market_share, left_on='state', right_on='SubCode', how='left')#.drop('Attribute_x', axis=1)
market_sizes['Market Size']=market_sizes['sales_quantity'].div(market_sizes['Market Share'], axis=0)

#computing number of stores per state
x=sales_sum_df.groupby(['state','period'])['store_type'].unique()
l=[]
store_nos=pd.DataFrame()
for i in range(len(x)):
    l.append([x.index[i][0],x.index[i][1],len(x[i])])
cols=['state','period','Store numbers']
store_nos = pd.DataFrame(l, columns=cols)

#merging market sizes with number of stores per market
market_sizes=pd.merge(market_sizes,store_nos, how='inner')

#computing market size per store
market_sizes['per store market']=market_sizes['Market Size']/market_sizes['Store numbers']

#adding market share per store-period to the main data
market_sizes=market_sizes[['state','period','per store market']]#extracting only relevant columns from market_sizes
merge_cols=['state','period']
summary_with_market_shares=pd.merge(sales_sum_df,market_sizes, on=merge_cols,how='inner')

#computing So

# summary_with_market_shares['so'] = summary_with_market_shares['per store type market']-summary_with_market_shares['sales_quantity']

summary_with_market_shares['so']=summary_with_market_shares['per store market']-summary_with_market_shares['sales_quantity']
summary_with_market_shares = summary_with_market_shares.fillna(0)[summary_with_market_shares['so'] != 0]

#computing log(S/So) [replacing zeros with 1e-08 so that logs dont create a problem]
summary_with_market_shares['so']=summary_with_market_shares['sales_quantity'].div(summary_with_market_shares['so'],axis=0)
summary_with_market_shares['so'] = summary_with_market_shares['so'].replace(0,10**(-5))

summary_with_market_shares['so']=np.log(summary_with_market_shares['so'])

In [13]:
# 2(b)(ii) checking for NaNs
d=summary_with_market_shares[['sales_quantity','per store market','so']]
d[d.isna().any(axis=1)]

,sales_quantity,per store market,so


In [14]:
#2(b)(iii)checking for inf values
d.iloc[d.index[np.isinf(d).any(1)]]

,sales_quantity,per store market,so


In [15]:
#2(b)(iii) checking for inf values
[summary_with_market_shares['period'].iloc[d.index[np.isinf(d).any(1)]].value_counts()]

[Series([], Name: period, dtype: int64)]

In [16]:
#2(b)(iv) extracting specific columns

col=[ 'item_no','period', 'state', 'region',
       'brand', 'stock_prevailing_mrp', 'store_type', 'store_location', 'city_type', 'case_size_range',
       'gender', 'movement', 'material', 'dial_color', 'strap_type',
       'strap_color', 'precious_stone', 'glass', 'case_shape', 'watch_type','billing','sales_quantity','so']

summary_final=summary_with_market_shares.loc[:,col]
#df_north_final.fillna(0, inplace=True)

summary_final['item_no']=summary_final['item_no'].astype(str)
summary_final['period']=summary_final['period'].astype(int)
summary_final['case_shape']=summary_final['case_shape'].astype(str)

In [17]:
summary_final = summary_final[summary_final['period'] != 14]


### 2c. Performing Feature Pareto

#### Defining a function for doing pareto analysis on features

The function combines all levels of a categorical features that cummulatively account for ~10% or less by Sales billings into a new level called "others". Features with less than 10 levels are not considered for pareto analysis.

In [18]:
def pareto(df,cols):
    lst=[]
    feature_mapping={}
    for col in cols:
                
        series=df.fillna(0).groupby([col]).agg({'billing':'sum'}).sort_values('billing',ascending=False)
        mask=series.cumsum()/series.sum()>0.9 
        mask=mask.iloc[:,0]
        levels=len(df[col].unique())
        
        if levels>10:
            df[col] = np.where(df[col].isin(series[mask].index),'Other',df[col])
            feature_mapping[col]=series[mask].index.to_list()
        new_levels=len(df[col].unique())
                       
        freq=df[col].value_counts()/df[col].value_counts().sum()*100
        freq=freq.round(2)

        sale_qty=df.groupby([col]).agg({'sales_quantity':'sum'}).sort_values('sales_quantity',ascending=False)
        sale_qty=sale_qty/sale_qty.sum()*100
        sale_qty=sale_qty.round(2)
        try:
            Other_Sales_Qty=sale_qty['sales_quantity']['Other']
        except:
            Other_Sales_Qty=0
        
        bill=df.groupby([col]).agg({'billing':'sum'}).sort_values('billing',ascending=False)
        bill=bill/bill.sum()*100
        bill=bill.round(2)
        try:
            Other_bill=bill['billing']['Other']
        except:
            Other_bill=0
        
        #comparison=mrp.merge(sale_qty, left_index=True, right_index=True)
        lst.append([col.upper(),levels, new_levels,Other_bill,Other_Sales_Qty])
        #print ("%s-Originally %d levels,combined %d levels into 'Other'.New Levels %d.By MRP,Other is %2.1f and by sale qty others is %2.1f"%(col.upper(),levels, levels-new_levels, new_levels,mrp['stock_prevailing_mrp']['Other'],sale_qty['sales_quantity']['Other']))
    
    cols=['Feature', 'Orig Levels', 'New Levels', 'Other%(Billing)', 'Other%(Sales Qty)']
    df1 = pd.DataFrame(lst, columns=cols)
    df1=df1.set_index("Feature")
    
    return df1,df, feature_mapping

In [19]:
import numpy as np
#cols=['brand','case_size', 'case_size_range', 'gender', 'material', 'dial_color', 'strap_type', 'strap_color','precious_stone', 'glass', 'watch_type']
cols=['case_size_range', 'gender','movement', 'material', 'dial_color', 'strap_type', 'strap_color','precious_stone', 'glass', 'case_shape', 'watch_type']

#cols=['case_size_range']
summary,summary_final_pareto,feature_mapping_dict=pareto(summary_final, cols)
summary

,Orig Levels,New Levels,Other%(Billing),Other%(Sales Qty)
Feature,,,,
CASE_SIZE_RANGE,13,7,11.03,15.61
GENDER,3,3,0.00,0.00
MOVEMENT,6,6,0.00,0.00
MATERIAL,56,7,11.96,14.47
DIAL_COLOR,46,9,11.28,10.44
STRAP_TYPE,60,8,11.57,16.31
STRAP_COLOR,44,7,11.04,24.60
PRECIOUS_STONE,8,8,0.00,0.00
GLASS,7,7,0.00,0.00


## 3. Modelling

### 3a. Building a Classifier

In [20]:
#retaining only 0 and 1 for sales_quantity
summary_final_classifier = pd.DataFrame()
summary_final_classifier = pd.concat([summary_final_classifier, summary_final_pareto])
summary_final_classifier['sales_class'] = np.where(summary_final_classifier['sales_quantity'] > 0,True,False)         
#summary_final_classifier['sales_quantity'].value_counts().sort_index()
#summary_final_classifier.columns

In [21]:
summary_final_classifier.columns

Index(['item_no', 'period', 'state', 'region', 'brand', 'stock_prevailing_mrp',
       'store_type', 'store_location', 'city_type', 'case_size_range',
       'gender', 'movement', 'material', 'dial_color', 'strap_type',
       'strap_color', 'precious_stone', 'glass', 'case_shape', 'watch_type',
       'billing', 'sales_quantity', 'so', 'sales_class'],
      dtype='object')

In [22]:
#creating dummy variables
cols=['brand','state','region', 'store_type', 'store_location', 'city_type',
       'case_size_range', 'gender', 'movement', 'material', 'dial_color',
       'strap_type', 'strap_color', 'precious_stone', 'glass', 'case_shape',
       'watch_type']
summary_final_classifier_dummies=pd.get_dummies(data=summary_final_classifier, columns=cols)

#for sales_quantity as the target variable
#creating seperate df for independent and dependent features
y_clf=summary_final_classifier_dummies.loc[:, summary_final_classifier_dummies.columns == 'sales_class']
X_clf=summary_final_classifier_dummies.drop(columns =['sales_quantity','item_no','billing','so', 'sales_class'])

#performing train and test split on data
X_train_clf,X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)

In [23]:
#checking for duplicate column names
duplicate_columns = summary_final_classifier_dummies.columns[summary_final_classifier_dummies.columns.duplicated()]
duplicate_columns

Index([], dtype='object')

In [19]:
#sorting by week_no and year
#df_north_pareto=df_north_pareto.sort_values(['week_no', 'year'], ascending=True)

In [20]:
#creating dummy variables
#cols=['brand','state','region', 'store_type', 'store_location', 'city_type',
#       'case_size_range', 'gender', 'movement', 'material', 'dial_color',
#       'strap_type', 'strap_color', 'precious_stone', 'glass', 'case_shape',
#       'watch_type', 'week_no', 'year']
#df_north_dummies=pd.get_dummies(data=df_north_pareto, columns=cols)

In [21]:
#for sales_quantity as the target variable
#creating seperate df for independent and dependent features
#y=df_north_dummies.loc[:, df_north_dummies.columns == 'sales_quantity']
#X=df_north_dummies.drop(columns =['available_quantity','sales_quantity','location_code','item_no','billing','so'])

In [22]:
#for So as the target variable
#creating seperate df for independent and dependent features
#y=df_north_dummies.loc[:, df_north_dummies.columns == 'so']
#X=df_north_dummies.drop(columns =['available_quantity','sales_quantity','location_code','item_no','billing','so'])

In [23]:
#performing train and test split on data

#split=0.2
#test=int(len(X)*split)
#train=len(X)-test
#X_train=X.head(train)
#y_train=y.head(train)
#X_test=X.tail(test)
#y_test=y.tail(test)

In [24]:
#creating dummy variables
#cols=['brand','state','region', 'store_type', 'store_location', 'city_type',
#       'case_size_range', 'gender', 'movement', 'material', 'dial_color',
#       'strap_type', 'strap_color', 'precious_stone', 'glass', 'case_shape',
#       'watch_type', 'week_no', 'year']
#df_north_dummies_clf=pd.get_dummies(data=df_north_pareto_classifier, columns=cols)

#for sales_quantity as the target variable
#creating seperate df for independent and dependent features
#y_clf=df_north_dummies_clf.loc[:, df_north_dummies_clf.columns == 'sales_quantity']
#X_clf=df_north_dummies_clf.drop(columns =['sales_quantity','location_code','item_no','billing','so'])

#performing train and test split on data

#split=0.2
#test_clf=int(len(X_clf)*split)
#train_clf=len(X_clf)-test_clf
#X_train_clf=X_clf.head(train_clf)
#y_train_clf=y_clf.head(train_clf)
#X_test_clf=X_clf.tail(test_clf)
#y_test_clf=y_clf.tail(test_clf)

#### Balancing the data

In [24]:
#!pip install -U imbalanced-learn
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=11915027)
X_bal_clf, y_bal_clf = ros.fit_resample(X_train_clf, y_train_clf)
#classifier(X_bal,y_bal)

Using TensorFlow backend.


In [25]:
print (len(X_train_clf),len(X_bal_clf),len(y_train_clf),len(y_bal_clf))

312123 447014 312123 447014


In [26]:
counts=y_train_clf.iloc[:, 0].value_counts().to_frame(name='Before Balancing')
count1= pd.DataFrame(np.bincount(y_bal_clf.iloc[:, 0]))
counts['After Balancing'] = count1
counts

,Before Balancing,After Balancing
False,223507,223507
True,88616,223507


#### Defining function for computing confusion metrics

In [27]:
from sklearn import metrics

In [28]:
#defining function for computing confusion metrics

from sklearn import metrics
from sklearn.metrics import confusion_matrix
def cfm(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    tnp=tn/(tn+fp)*100
    fpp=fp/(tn+fp)*100
    fnp=fn/(fn+tp)*100
    tpp=tp/(fn+tp)*100
    false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_test, y_pred)
    roc_auc = metrics.auc(false_positive_rate, true_positive_rate)
    accuracy=metrics.accuracy_score(y_test, y_pred)*100
    return accuracy,tnp,fpp,fnp,tpp,roc_auc
    #return accuracy,tn,fp,fn,tp

### 3b. Regression model on non-zero sales data

In [33]:
#Slicing for data with 1s as target variable
summary_final_pareto_reg = summary_final_pareto[summary_final_pareto['sales_quantity'] > 0]

In [34]:
#creating dummy variables
cols=['brand','state','region', 'store_type', 'store_location', 'city_type',
       'case_size_range', 'gender', 'movement', 'material', 'dial_color',
       'strap_type', 'strap_color', 'precious_stone', 'glass', 'case_shape',
       'watch_type']
summary_final_dummies=pd.get_dummies(data=summary_final_pareto_reg, columns=cols)

print('Done')

Done


In [35]:
summary_final_dummies.shape

(110602, 191)

In [36]:
#checking for duplicate column names
duplicate_columns = summary_final_dummies.columns[summary_final_dummies.columns.duplicated()]
duplicate_columns

Index([], dtype='object')

In [37]:
#for So as the target variable
#creating seperate df for independent and dependent features
y=summary_final_dummies.loc[:, summary_final_dummies.columns == 'so']
X=summary_final_dummies.drop(columns =['sales_quantity','item_no','billing', 'so'])

In [38]:
#performing train and test split on data
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


"""
split=0.2
test=int(len(X)*split)
train=len(X)-test
X_train=X.head(train)
y_train=y.head(train)
X_test=X.tail(test)
y_test=y.tail(test)
"""

'\nsplit=0.2\ntest=int(len(X)*split)\ntrain=len(X)-test\nX_train=X.head(train)\ny_train=y.head(train)\nX_test=X.tail(test)\ny_test=y.tail(test)\n'

In [ ]:
#best model based on the output of random_search.best_estimator_
best_gb=xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=12,
             min_child_weight = 7, missing=np.nan,
              n_estimator=3, n_estimators=100,
             n_jobs=0, num_parallel_tree=1, objective='reg:squarederror',
             random_state=0, reg_alpha=0.5, reg_lambda=0.3, scale_pos_weight=1,
             subsample=0.2, tree_method='exact', validate_parameters=1,
             verbosity=0)

In [40]:
#running the regression model
from sklearn.linear_model import LinearRegression
reg_model_so = LinearRegression()  
reg_model_so.fit(X_train, y_train)
preds_so = reg_model_so.predict(X_test)
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
rmse = np.sqrt(mean_squared_error(y_test, preds_so))
r2=r2_score(y_test, preds_so)
print("Linear regression RMSE: %.2f, Test R2: %.2f" % (rmse,r2))

Linear regression RMSE: 0.49, Test R2: 0.67


## 4.  Predictions on data

In [130]:
#defining the store, brand and period for which the assortment is being developed
store = 'S28'
period=14
brand='B063'

In [131]:
#obtaining unique SKUs
cols_for_unique_selection = ["item_no","brand","case_size_range","gender","movement","material","dial_color","strap_type",
        "strap_color","precious_stone","glass","case_shape","watch_type"]
                             
unique_sku=summary_df[cols_for_unique_selection].drop_duplicates(cols_for_unique_selection)

In [132]:
#replace features from pareto
cols=['case_size_range', 'material', 'dial_color', 'strap_type', 'strap_color']
for col in cols:
    unique_sku[col] = np.where(unique_sku[col].isin(feature_mapping_dict[col]),'Other',unique_sku[col])

In [133]:
#replace items with Others - replace brands first
unique_sku['brand'] = np.where(unique_sku['item_no'].isin(item_mapping_list),'Others',unique_sku['brand'])
unique_sku['item_no'] = np.where(unique_sku['item_no'].isin(item_mapping_list),'Others',unique_sku['item_no'])

In [134]:
#adding latest MRPs
item_mrp = pd.read_excel("D:\\Backup\\ISB CBA\\Capstone\\itemMRP_encoded.xlsx")
item_mrp['item_no'] = np.where(item_mrp['item_no'].isin(item_mapping_list),'Others',item_mrp['item_no'])
item_mrp['brand'] = np.where(item_mrp['item_no'].isin(item_mapping_list),'Others',item_mrp['brand'])
item_mrp = item_mrp.rename(columns={"MRP": "stock_prevailing_mrp"})
item_mrp_agg = item_mrp.groupby(['item_no', 'brand']).agg({'stock_prevailing_mrp':'mean'}).reset_index()
unique_sku = unique_sku.merge(item_mrp_agg, left_on=['item_no'], right_on=['item_no'], how='left')

In [135]:
#checking for nulls
item_mrp_agg.columns[item_mrp_agg.isna().any()].tolist()

[]

In [136]:
unique_sku=unique_sku.drop(['brand_x'],axis=1)
unique_sku.rename(columns={'brand_y': 'brand'}, inplace=True)
unique_sku.columns

Index(['item_no', 'case_size_range', 'gender', 'movement', 'material',
       'dial_color', 'strap_type', 'strap_color', 'precious_stone', 'glass',
       'case_shape', 'watch_type', 'brand', 'stock_prevailing_mrp'],
      dtype='object')

In [137]:
#to be removed
unique_sku.columns[unique_sku.isna().any()].tolist()

['brand', 'stock_prevailing_mrp']

In [138]:
#to be removed
unique_sku = unique_sku.fillna(0)

In [139]:
#creating dummy variables
cols=['brand','case_size_range', 'gender', 'movement', 'material', 'dial_color',
       'strap_type', 'strap_color', 'precious_stone', 'glass', 'case_shape',
       'watch_type']
unique_sku_dummies=pd.get_dummies(data=unique_sku, columns=cols)

In [140]:
#adding store dummies
store_master = pd.read_csv("D:\\Backup\\ISB CBA\\Capstone\\store_master_with_state_and_regions.csv")

In [141]:
#adding store dummies
store_dummies = [
    'state_ST02', 'state_ST03', 'state_ST05', 'state_ST07', 'state_ST09', 'state_ST12', 'state_ST13', 'state_ST15', 'state_ST16', 
    'state_ST17', 'state_ST19', 'state_ST21', 'state_ST24', 'state_ST28', 'region_East', 'region_North', 'region_South',
'region_West', 'store_type_Ethos', 'store_type_Large', 'store_type_Summit', 'store_location_Airport', 'store_location_DFS', 
    'store_location_High Street', 'store_location_Mall', 'city_type_Tier1', 'city_type_Tier2'
]

for col in store_dummies:
    unique_sku_dummies[col] = 0
    
#Modifying dummy values for store attributes
store_master = pd.read_csv("D:\\Backup\\ISB CBA\\Capstone\\store_master_with_state_and_regions.csv")
store_cols = ["state","region","store_type","store_location","city_type"]
for col in store_cols:
    dummy_col_name = col + '_' + store_master[store_master['store_code'] == store][col].unique()[0]
    print(dummy_col_name)
    unique_sku_dummies[dummy_col_name] = 1

state_ST12
region_West
store_type_Ethos
store_location_Mall
city_type_Tier1


In [142]:
unique_sku_dummies.shape

(160316, 204)

In [143]:
#to check
unique_sku_dummies['brand_Others'] = 0
unique_sku_dummies['period'] = period

In [153]:
#checking that all dummy columns are created
X_data=unique_sku_dummies.drop(columns =['item_no'])
missing_dummies = set(X_train_clf) - set(X_data)
additional_dummies = set(X_data) - set(X_train_clf)
print ('No. of missing dummies: ', len(missing_dummies))
print ('No. of new dummies: ', len(additional_dummies))

No. of missing dummies:  0
No. of new dummies:  17


In [165]:
X_data=unique_sku_dummies.drop(columns =['item_no'])
X_data.shape

(160316, 205)

In [166]:
#removing the additional dummies
#missing_dummies_cols = list(missing_dummies_set)
for col in additional_dummies:
    X_data = X_data.drop(columns = [col])
X_data.shape

(160316, 188)

In [167]:
#Running the classifer model on the data
X_data=X_data[X_bal_clf.columns] # to get the column order as per training
y_pred= xgb_model.predict(X_data)

In [69]:
#appending classifier prediction to unique skus
#unique_sku_dummies['sales_class'] = y_pred
#unique_sku_pred = unique_sku
#unique_sku_pred['sales_class'] = y_pred
#unique_sku_pred['sales_class'].value_counts()

In [124]:
#Obtaining items that are predicted for sales
#items_classified_for_sales = unique_sku_dummies[unique_sku_dummies['sales_class']]['item_no'].unique()
#len(items_classified_for_sales)

In [168]:
#how many SKUs predicted as non zero
unique, counts = np.unique(y_pred, return_counts=True)
print (np.asarray((unique, counts)).T)

[[     0 160315]
 [     1      1]]


In [169]:
#slicing data for non-zero predictions (to run the regression model)
X_data_non_zero=X_data[y_pred==1]

In [170]:
X_data_non_zero

,period,stock_prevailing_mrp,brand_B007,brand_B008,brand_B010,brand_B013,brand_B014,brand_B017,brand_B018,brand_B020,...,case_shape_rectangle,case_shape_round,case_shape_square,case_shape_tonneau,watch_type_activity tracker,watch_type_analog,watch_type_analog-digital,watch_type_digital,watch_type_hybrid smart watch,watch_type_smart watch
2676,14,349600.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [93]:
# Applying regression model on items that are predicted to sell


#unique_sku_dummies_so = unique_sku_dummies[unique_sku_dummies['item_no'].isin(items_classified_for_sales)]
#unique_sku_dummies_so = unique_sku_dummies[unique_sku_dummies['item_no'].isin(items_classified_for_sales)]
#X_data_so = unique_sku_dummies_so.drop(columns =['item_no'])
#X_data_so = unique_sku_dummies.drop(columns =['item_no'])#this line to be removed
#X_data_so.shape

In [94]:
#missing_dummies_set = set(X_data_so) - set(X_train2)
#print(missing_dummies_set)

In [95]:
#X_data_so = X_data_so.drop(columns = list(missing_dummies_set))

In [96]:
#missing_dummies_set = set(X_train2) - set(X_data_so)
#print(missing_dummies_set)

In [115]:
pred_non_zero = reg_model_so.predict(X_data_non_zero)
predictions_for_store = unique_sku_pred[unique_sku_pred['sales_class']]
predictions_for_store['so'] = np.concatenate( pred_non_zero, axis=0 ).tolist()

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 187 is different from 188)

In [77]:
predictions_for_store['so']=

SyntaxError: invalid syntax (<ipython-input-77-dc0abbbf6f37>, line 1)

## 5. Creating the Assortment

In [ ]:
#Slice prediction for store and brand



In [ ]:
####To change

#Compute profit per SKU of the brand for the store
profit=pd.read_excel ("D:\\Backup\\ISB CBA\\Capstone\\profit_computation_S28.xlsx", sheet_name='Sheet1',header=0)
profit=profit[['item_no','Net Profit per unit']]
profit=profit.set_index('item_no')
profit.head()

In [ ]:
## this code to be used when actual predictions are available, Till then use random predictions in the next cell


#predicted log S/So
#pred_log_s=pd.read_csv("D:\\Backup\\ISB CBA\\Capstone\\greedy_input.csv")
#pred_log_s=pred_log_s[['SKU','Ln (S/ So)']]
#pred_log_s=pred_log_s.set_index('SKU')
#pred_log_s['exp'] = np.exp(pred_log_s['Ln (S/ So)'])
#pred_log_s['S']=market_size/(1+pred_log_s['exp'])*pred_log_s['exp']
#pred_log_s=pd.merge(pred_log_s,profit,left_index=True,right_index=True)
#pred_log_s

In [ ]:
# generating random predictions. Using the item MRP file for obtaining the unique SKUs

pred_log_s= pd.read_excel("D:\\Backup\\ISB CBA\\Capstone\\itemMRP_encoded.xlsx")
pred_log_s['Ln (S/ So)']=np.random.rand(len(pred_log_s))*(-20)# random predictions
pred_log_s=pred_log_s.drop(['MRP'],axis=1)#deleting MRPs
pred_log_s=pred_log_s.set_index('item_no')
pred_log_s['exp'] = np.exp(pred_log_s['Ln (S/ So)'])# taking exponent
pred_log_s['S']=market_size/(1+pred_log_s['exp'])*pred_log_s['exp']#calculating predicted sale qty
pred_log_s=pd.merge(pred_log_s,profit,left_index=True,right_index=True)#merging with profit df to get profit column. 
                                                                       #Also helps in culling dthe unique SKUs to only 
                                                                       #those which are retailed in specific store
pred_log_s

In [ ]:
#Define greedy algorthim function
def assortment (predictions, size):
    assortment_size=size
    for i in range(assortment_size):
        i+=1
        #print (i)
        best_profit=0
        if i==1:
            best_selection=pd.DataFrame(columns=predictions.columns)
        #print (predictions.index)
        for sku in predictions.index:
            #print (sku)
            testing=best_selection
            if sku not in testing.index:
                row=predictions.loc[sku]
                testing=testing.append(row)
                testing['S']=market_size/(1+testing['exp'].sum())*predictions['exp']
                testing['Profit']=testing['S'].mul(testing['Net Profit per unit'],axis=0)
                if best_profit<=testing['Profit'].sum():
                    best_profit=testing['Profit'].sum()
                    candidate=testing
            else:
                continue
        best_selection=candidate
    #return best_selection['Profit'].sum().round(0)
    best_selection=best_selection.reset_index()
    best_selection.loc['Total']= best_selection.sum()
    return best_selection

In [ ]:
#creating an assortment for 50 skus for brand B063
pred_log_s_brand=pred_log_s[pred_log_s['brand']==brand]
assortment(pred_log_s_brand, 50)